In [1]:
import pandas as pd
import json

import nltk
import pandas as pd
import numpy as np
import re
import time
import sys
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from pprint import pprint
from functools import reduce
from matplotlib import pyplot as plt
import string

import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import KFold
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
from tqdm import tqdm
from tabulate import tabulate
import argparse

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from xgboost import XGBClassifier

C:\Users\navvu\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
temp = pd.read_csv('For_preprocessing.csv')
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4768 entries, 0 to 4767
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review_Text  4763 non-null   object
 1   Sentiment    4766 non-null   object
dtypes: object(2)
memory usage: 74.6+ KB


In [3]:
temp

,Review_Text,Sentiment
0,Its really nice place to stay especially for b...,3
1,It seems that hotel does not check the basic a...,1
2,Worst hotel I have ever encountered. I will ne...,1
3,Had a good time in this hotel and the staff Ku...,3
4,good hotel and staff Veg food good non veg bre...,3
...,...,...
4763,My fifth stay at the hotel for business. Rooms...,3
4764,enjoyable,3
4765,Most impressive service by staff in all areas....,3
4766,"The linens were smelling bad, and the elevator...",1


In [4]:
temp=temp.dropna(subset=['Review_Text'])
temp=temp.dropna(subset=['Sentiment'])
temp = temp.reset_index(drop=True)
temp_val = temp['Sentiment'].tolist()
#len(temp_val)
ind_list = []

for i in range(len(temp['Sentiment'])):
    if temp_val[i] != '1' and temp_val[i] != '2' and temp_val[i] != '3':
        ind_list.append(i)
temp.reset_index(drop=True)
temp.drop(temp.index[ind_list], inplace=True)

In [5]:
type(temp['Sentiment'][0])

str

In [6]:
temp = temp.reset_index(drop=True)
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4699 entries, 0 to 4698
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review_Text  4699 non-null   object
 1   Sentiment    4699 non-null   object
dtypes: object(2)
memory usage: 73.5+ KB


In [7]:
# def text_processing(tweet):
    
#     #Generating the list of words in the tweet (punctuations removed)
#     def form_sentence(tweet):
#         tweet_blob = TextBlob(tweet)
#         return ' '.join(tweet_blob.words)
#     new_tweet = form_sentence(tweet)
    
#     #Removing stopwords and words with unusual symbols
#     def no_user_alpha(tweet):
#         tweet_list = [ele for ele in tweet.split()]
#         clean_tokens = [t for t in tweet_list if re.match(r'[^\W\d]*$', t)]
#         clean_s = ' '.join(clean_tokens)
#         clean_mess = [word for word in clean_s.split() if word.lower() not in stopwords.words('english')]
#         return clean_mess
#     no_punc_tweet = no_user_alpha(new_tweet)
    
#     #Normalizing the words in tweets 
#     def normalization(tweet_list):
#         lem = WordNetLemmatizer()
#         normalized_tweet = []
#         for word in tweet_list:
#             normalized_text = lem.lemmatize(word,'v')
#             normalized_tweet.append(normalized_text)
#         return normalized_tweet
    
    
#     return normalization(no_punc_tweet)

In [8]:
def tokenize_reg():
    temp['Review_Text'] = temp['Review_Text'].replace(r'http\S+','',regex=True).replace(r'www\S+','',regex=True).replace(r'\d+','',regex=True)
    tokens = RegexpTokenizer(r'\w+')
    temp['Review_Text']=temp['Review_Text'].apply(lambda x:tokens.tokenize(x.lower()))
    stop_words = set(stopwords.words('english')) 
    stop_words = stop_words.union(",","(",")","[","]","{","}","#","@","!",":",";",".","?")
    temp['Review_Text'] = temp['Review_Text'].apply(lambda x: [item for item in x if item not in stop_words])
tokenize_reg()

In [9]:
lem = WordNetLemmatizer()
def lemm(text):
    sent=[]
    for word in text:
        sent.append(lem.lemmatize(word))
    return sent

ps=PorterStemmer()

def stemm(text):        
    sent = []
    for word in text:
        sent.append(ps.stem(word))
    return sent

temp['Review_Text'] =  temp.apply(lambda x: stemm(lemm(x['Review_Text'])), axis=1)

In [10]:
revs=temp['Review_Text'].tolist()
for i in range(len(revs)):
    revs[i]=' '.join(revs[i])
temp['Review_Text'] = revs
temp

,Review_Text,Sentiment
0,realli nice place stay especi busi tourist purpos,3
1,seem hotel check basic amen room hand room tra...,1
2,worst hotel ever encount never think stay thii...,1
3,good time hotel staff kumar aishwarya hous kee...,3
4,good hotel staff veg food good non veg breakfa...,3
...,...,...
4694,fifth stay hotel busi room great restaur excel...,3
4695,enjoy,3
4696,impress servic staff area good restaur fit cen...,3
4697,linen smell bad elev pungent odour housekeep p...,1


In [11]:
st = temp['Sentiment'].tolist()
st=set(st)

In [12]:
st

{'1', '2', '3'}

In [13]:
# countVec = CountVectorizer(analyzer=text_processing)
# countVec.fit(temp["Review_Text"])
# list(countVec.vocabulary_.items())

In [14]:
# countVec_count = countVec.transform(temp["Review_Text"])

# #make array from number of occurrences
# occ = np.asarray(countVec_count.sum(axis=0)).ravel().tolist()

# #make a new data frame with columns term and occurrences, meaning word and number of occurences
# bowListFrame = pd.DataFrame({'term': countVec.get_feature_names(), 'occurrences': occ})
# print(bowListFrame)

# #sort in order of number of word occurences, most->least. if you leave of ascending flag should default ASC
# bowListFrame.sort_values(by='occurrences', ascending=False).head(60)


In [15]:
# textTransformer = TfidfTransformer()
# textWeights = textTransformer.fit_transform(countVec_count)

In [16]:
# #follow similar process to making new data frame with word occurrences, but with term weights
# textWeightsFin = np.asarray(textWeights.mean(axis=0)).ravel().tolist()

# #now that we've done Tfid, make a dataframe with weights and names
# textWeightFrame = pd.DataFrame({'term': countVec.get_feature_names(), 'weight': textWeightsFin})
# print(textWeightFrame)
# textWeightFrame.sort_values(by='weight', ascending=False).head(20)

In [17]:
X = temp['Review_Text']
y = temp['Sentiment']

#test = test_tweets['post']
from sklearn.model_selection import train_test_split
msg_train, msg_test, label_train, label_test = train_test_split(temp['Review_Text'], temp['Sentiment'], test_size=0.2)

In [21]:
import xgboost as xgb
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

In [22]:
def objective(space):
    
    clf=Pipeline([
    ('bow',CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),
    xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    ])
    evaluation = [( msg_train, label_train), ( msg_test, label_test)]
    
    clf.fit(msg_train, label_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(msg_test)
    accuracy = accuracy_score(test_y, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [23]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)


  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

job exception: 'XGBClassifier' object is not iterable



  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]


TypeError: 'XGBClassifier' object is not iterable

In [ ]:
# !pip install "gensim==3.8.3"

In [ ]:
from gensim.sklearn_api import W2VTransformer

In [ ]:
!pip install hyperopt==0.2.5

In [ ]:
# HyperOpt-Sklearn 
from hpsklearn import HyperoptEstimator
from hpsklearn import any_classifier
from hpsklearn import any_preprocessing

# HyperOpt: Optimizer
from hyperopt import tpe

In [ ]:
#Machine Learning Pipeline

pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=text_processing)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', HyperoptEstimator(classifier=any_classifier('cla'), 
                          preprocessing=any_preprocessing('pre'), 
                          algo=tpe.suggest, 
                          max_evals=10)),  # train on TF-IDF vectors w/ Naive Bayes classifier
])
pipeline.fit(msg_train,label_train)

#fmin(f_lgbm, lgbm_param, algo=tpe.suggest, max_evals=MAX_EVAL, trials=trials, rstate=np.random.default_rng(SEED))

In [ ]:
predictions = pipeline.predict(msg_test)

# print("Classification report")
# print(classification_report(predictions,label_test))
print ("Confusion Matrix")
print(confusion_matrix(predictions,label_test))
print("Accuracy Score")
print(accuracy_score(predictions,label_test))

In [ ]:
print("Classification report")
print(classification_report(predictions,label_test))

In [ ]:
df = pd.read_csv('kolkata_reviews.csv',encoding='utf-8')
df

In [ ]:
predictions = pipeline.predict(df['text'])

In [ ]:
df['Sentiment']=predictions
df

In [ ]:
df['Sentiment'].value_counts()